# Project : Recipe Recommendations Based on Grocery Store Promotions

#### Notebook 3: 
In Notebook 1 and Notebook 2, two CSV files were saved, "recipes_clean_v2.csv" and "grocerydeals_v3.csv", respectively. Using these CSV files, recipes are generated by randomly generating a list of 5 grocery store items. Additionally, the functions created for this project are saved as pickle files (for deployment on Flask).

-----------------------------------------------------------------------------------------------------------------------------------------------------------

### 1. Imports

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import numpy as np
import pickle

### 2. Obtain the Two Cleaned Dataframes

- 'recipes_clean_v2.csv' : recipe database from the Food Network
- 'grocerydeals_v3.csv' : food deals from grocery stores (Super C, Metro)

In [31]:
df_recipes=pd.read_csv('recipes_clean_v2.csv')
df_deals=pd.read_csv('grocerydeals_v3.csv')

In [32]:
len(df_recipes)

8080

In [33]:
len(df_deals)

2454

### 3. Recipe Recommendation

#### 3.1 Creating a Function to Generate a List of Random Grocery Store Items

In [3]:
def random_groceries_5():
    # Created 5 categories of grocery types based on the aisle categories
    category1=["Beverages","Beer & Wine"]
    category2=["Frozen","Deli-Prepared","Pantry", "Snacks"]
    category3=["Meat & Poultry", "Fish & Seafood"]
    category4=["Fruits & Vegetable", "Organic Groceries"]
    category5=["Vegan & Vegetarian","Dairy & Eggs","Bread & Bakery Prodructs","World Cuisine"]

    df_cat1=df_deals[df_deals['Aisle'].isin(category1)]
    df_cat2=df_deals[df_deals['Aisle'].isin(category2)]
    df_cat3=df_deals[df_deals['Aisle'].isin(category3)]
    df_cat4=df_deals[df_deals['Aisle'].isin(category4)]
    df_cat5=df_deals[df_deals['Aisle'].isin(category5)]

    random_item_cat1=df_cat1.sample(n=1, replace=False)
    random_item_cat2=df_cat2.sample(n=1, replace=False)
    random_item_cat3=df_cat3.sample(n=1, replace=False)
    random_item_cat4=df_cat4.sample(n=1, replace=False)
    random_item_cat5=df_cat5.sample(n=1, replace=False)

    random_groceries=pd.concat([random_item_cat1, random_item_cat2, random_item_cat3, random_item_cat4, random_item_cat5], ignore_index=True)
    return random_groceries

#### 3.2 Creating a Function to Generate a List of Recipes Based on the Grocery Store Items

In [35]:
def recipe_recommendations():
    
    recommendations=[]
    
    ### Step 1. Set-Up TfidfVectorizer
    tf=TfidfVectorizer()

    ### Step 2. Transform the grocery store items to vectors

    # 1. All the words from "Product_Clean" from the 5 grocery items to be analysed together
    groceries=random_groceries_5()
    #print(groceries)
    
    groceries_=pd.DataFrame(groceries[['Grocery Store','Product','Aisle','Brand','Deal Price','Promotion Effectivity']])
    #print(groceries_)
    
    # 2. Vectorize the combined_groceries
    combined_groceries=groceries['Product_Clean'].str.cat(sep=" ")
    X_groceries=tf.fit_transform([combined_groceries])

    keywords=tf.get_feature_names_out() #word associated to each vector
    #print(keywords)

    ### Step 3. Transform the ingredients from recipe into vectors
    X_recipe=tf.transform(df_recipes['Ingredients_Cleaned'])

    ### Step 4. Recipe Closest in Value with Grocery Items
    scores = cosine_similarity(X_groceries, X_recipe)
    #scores.shape # 1,8080
    #print(scores.max())

    ### Step 5. Return 5 Recipes
    top_5_recipes_index=np.argsort(scores)[:, -5:][:, ::-1]

    top_5_recipes_index=top_5_recipes_index[0]
    for i in top_5_recipes_index:
        recommendations.append(df_recipes.iloc[i][['Chef','Recipe Title','Level of Difficulty','Total Time Required','Ingredients','Number of Ingredients in the Recipe','Link to Recipe']])

    pd.set_option('display.max_colwidth', None)
    df_recommendations=pd.DataFrame(recommendations)
    return groceries_,df_recommendations

In [36]:
results=recipe_recommendations()

In [37]:
results[0]

,Grocery Store,Product,Aisle,Brand,Deal Price,Promotion Effectivity
0,Metro,"Light Lager Beer, Ultra",Beer & Wine,Molson,2 /$62.98,"Valid until Dec 6, 2023"
1,Metro,"Churros Flavoured Cereal, Cinnamon Toast Crunch",Pantry,General Mills,$3.99 ea.,"Valid until Dec 6, 2023"
2,Metro,"Skin-On Chicken Thighs with Back, Value Pack",Meat & Poultry,The Item's Brand is Not Available,$9.61 avg.,"Valid until Dec 6, 2023"
3,Metro,Organic Avocados,Organic Groceries,The Item's Brand is Not Available,$2.99 ea.,"Valid until Dec 6, 2023"
4,Metro,Dairy-Free Plain Oat Based Yogurt Style,Vegan & Vegetarian,Silk,$4.99 ea.,"Valid until Dec 6, 2023"


In [38]:
results[1]

,Chef,Recipe Title,Level of Difficulty,Total Time Required,Ingredients,Number of Ingredients in the Recipe,Link to Recipe
7981,Recipe courtesy of Michael Symon,Grilled Chicken Thighs with Blackberry BBQ Sauce,Intermediate,3 hr,"['1 tablespoon ground coriander', '1 tablespoon smoked paprika', '1 tablespoon kosher salt', '8 skin-on, bone-in chicken thighs', 'Olive oil', '1/2 recipe Blackberry BBQ Sauce', '3 pints fresh blackberries', '12 ounces dark beer', '1 cup balsamic vinegar', '1 cup red wine vinegar', '1/2 cup packed light brown sugar', '1 onion, sliced', '1 clove garlic, minced', '1 habanero pepper, slit', '1 tablespoon ground chipotle chile powder', '1 tablespoon finely ground coffee', '1 tablespoon ground coriander', '1 tablespoon ground cumin']",18,https://www.foodnetwork.com/recipes/michael-symon/grilled-chicken-thighs-with-blackberry-bbq-sauce-5209285
1337,Recipe courtesy of Bobby Flay,"Spice-Rubbed Chicken Breast Tacos with Grilled Poblanos, BBQ Onions and Coleslaw",Intermediate,1 hr 25 min,"['3 large poblanos', 'Canola oil', 'Salt and freshly ground black pepper', '2 sweet onions, skin-on, and cut into 1/4-inch thick slices', 'Canola oil', 'Salt and freshly ground black pepper', ""1 cup Bobby Flay's Mesa Grill BBQ Sauce (or your favorite jarred variety)"", '3/4 cup mayonnaise', '2 tablespoons sugar', '1 teaspoon celery seed', '3 tablespoons apple-cider vinegar', 'Salt and freshly ground black pepper', '1 small head cabbage, cored, finely shredded', '2 carrots, finely shredded', '1/2 small red onion, thinly sliced', '3 scallions, thinly sliced on the diagonal', '2 tablespoons ancho chili powder', '2 teaspoons ground cinnamon', '2 teaspoons ground cumin', '2 teaspoons light brown sugar', '2 teaspoons kosher salt', '1/2 teaspoon ground black pepper', '4 boneless, skinless chicken breasts (8 ounces each)', 'Canola oil', '8 flour tortillas', '2 grilled sliced poblanos', 'Barbecued onions', 'Coleslaw', 'Guacamole, recipe follows', '2 ripe Hass avocados, peeled, pitted and diced', '1 poblano chile, roasted, seeded, peeled, and diced', '1/4 cup finely diced red onion', '2 scallions, finely chopped', '2 fresh limes', '1/2 cup chopped fresh cilantro leaves', '2 tablespoons canola oil', 'Salt and pepper']",37,https://www.foodnetwork.com/recipes/bobby-flay/spice-rubbed-chicken-breast-tacos-with-grilled-poblanos-bbq-onions-and-coleslaw-recipe-1953280
2563,Recipe courtesy of Bobby Flay,"Lamb and Black Bean Chili with Cumin Crema, Red Onion Relish, Avocado Relish and Native American Fry Bread",Level of Difficulty Not Available,Cooking Time Not Available,"['1/4 cup olive oil', '2 1/2 pounds lamb from shoulder, boned and cut into 1/2-inch cubes', 'Salt and freshly ground black pepper', '1 1/2 large Spanish onions, finely diced', '6 cloves garlic, finely chopped', '1 (15-ounce) can whole tomatoes, drained and pureed', '1 tablespoon chipotle puree', '3 tablespoons ancho chili powder', '1 tablespoon pasilla chili powder', '1 tablespoon ground cumin', '2 teaspoons ground coriander', '1 tablespoon dried Mexican oregano', '5 cups chicken stock', '1 (12-ounce) bottle dark beer', 'Dash ground cinnamon', '1 to 2 tablespoons honey', '2 cups cooked or canned black beans', 'Chopped cilantro leaves, for garnish', 'Cumin Crema, recipe follows', 'Avocado Relish, recipe follows', 'Red Onion Relish, recipe follows', 'Fry Bread, recipe follows', '1 pint creme fraiche, Mexican crema or sour cream', '1 tablespoon ground cumin', '1 tablespoon fresh lime juice', 'Salt and freshly ground pepper', '2 tablespoons canola oil', '2 red onions, finely diced', '1 tablespoon minced garlic', '1 serrano chile, finely diced with seeds', '1/4 cup freshly squeezed lime juice', 'Salt and pepper', '3 tablespoons finely chopped cilantro leaves', '4 Haas avocados, peeled, pitted and chopped', '1/2 cup red onion, minced', '1/2 cup chopped cilantro leaves', '2 serrano chiles, minced with seeds', '2 limes, juiced', 'Salt and pepper', '3 cups all-purpose flour', '1 tablespoo

### 4. Saving Functions for Flask
This step is required for the deployment of the project on Flask

In [39]:
# Saving the random_groceries function to a file 
with open('grocery_items.pkl', 'wb') as file:
    pickle.dump({'function':random_groceries_5, 'dataframe': df_deals}, file)

In [40]:
# # Verifying if it works
# with open('grocery_items.pkl', 'rb') as file:
#     saved_data=pickle.load(file)

# # Access the function and the DataFrame
# loaded_function=saved_data['function']
# loaded_dataframe=saved_data['dataframe']

In [42]:
# loaded_function()

In [43]:
# Saving the recipe_recommendation function to a file 
with open('recipe_recommendation.pkl', 'wb') as file:
    pickle.dump({'function':recipe_recommendations, 'dataframe': df_recipes}, file)

In [44]:
# # Verifying if it works
# with open('recipe_recommendation.pkl', 'rb') as file:
#     saved_data=pickle.load(file)

# # Access the function and the DataFrame
# loaded_function=saved_data['function']
# loaded_dataframe=saved_data['dataframe']

In [46]:
# loaded_function()

In [5]:
### CONVERT THIS NOTEBOOK INTO A .PY FILE
#!jupyter nbconvert --to python Notebook3_RecipeRecommendations.ipynb

[NbConvertApp] Converting notebook Notebook3_RecipeRecommendations.ipynb to python
[NbConvertApp] Writing 5343 bytes to Notebook3_RecipeRecommendations.py
